<a href="https://colab.research.google.com/github/danielamejiacastro/MLDS--Course/blob/main/Copia_de_Taller_2_Cassandra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://drive.google.com/uc?export=view&id=10rLzZzH5ctQ4bPRDZTbZk5tQMc9sDO0W" width="100%"></img>

# **Taller 2: Cassandra**
---

En este taller se evaluarán las habilidades adquiridas usando _Cassandra_ desde _Python_. Usted deberá realizar algunas operaciones de consulta sobre el conjunto de datos [Netflix Data: Cleaning, Analysis and Visualization](https://www.kaggle.com/datasets/ariyoomotade/netflix-data-cleaning-analysis-and-visualization)

Este conjunto de datos incluye la información acerca del contenido añadido a la plataforma de streaming *Netflix* entre el $2008$ y el $2021$. Está conformado por 12 columnas, las cuales son:

* `show_id:` corresponde a la llave primaria de la tabla. Tiene un formato establecido el cual corresponde a una <i>s</i> seguida de un número en secuencia, por ejemplo: s34.
* `type:` indica el tipo de show ofrecido (Película o Serie).
* `title:` señala el nombre de la serie o la película.
* `director:` indica el nombre de quién dirigió la película o serie.
* `country:` indica el lugar de producción del show.
* `date_added:` muestra la fecha de publicación de la serie o película en la plataforma con el formato <i>MMMMMM DD, AAAA</i>.
* `release_year:` muestra el año de publicación original de la película o serie.
* `rating:` muestra las calificaciones o el nivel de conveniencia de la película según su contenido, por ejemplo: PG-13, TV-PG, etc.
* `duration:` corresponde a la duración en minutos en el caso de las películas y la cantidad de temporadas en el caso de las series.
* `listed_in:` indica el género o categoría donde se clasifica la serie o película dentro de la plataforma.

> **Nota:** Esta tarea va a ser calificada **dentro del mismo notebook**. Para esto, en cada ejercicio se indicará las intrucciones, la función calificada, también los lugares donde debe escribir su código sin modificar lo demás con un aproximado de cantidad de líneas a escribir. No se preocupe si su código toma más líneas, esto es simplemente un aproximado destinado a que pueda replantear su estrategia si el código está tomando más de las esperadas. No es un requisito estricto y soluciones más largas también son válidas.

Al finalizar, para realizar el envío (*submission*), debe ejecutar la función `submit_task` al final, en la sección de evaluación **Edunext**. Recuerde que primero debe haber visto la actividad dentro de la plataforma para hacer correctamente el enlace y que la calificación se vea reflejada en esta.


Ejecute las siguientes celdas para crear el enlace con su cuenta de UNCode

In [2]:
!pip install rlxcrypt
!wget --no-cache -O session.pye -q https://raw.githubusercontent.com/JuezUN/INGInious/master/external%20libs/session.pye

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.9/297.9 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 43.8 MB/s eta 0:00:00
  Created wheel for imphook: filename=imphook-1.0-py3-none-any.whl size=9420 sha256=ee4d28c73cbeafcb55d30a2b584963f93805c653789cb3cd5f0fe9f322d0d20e
  Stored in directory: /root/.cache/pip/wheels/dc/e2/a4/fcb3817d09a2eb047b2b08eb58e7d9140041b0f3f415eb1256
Successfully built imphook


In [3]:
import rlxcrypt
import session

grader = session.LoginSequence("BD-GroupMLDS-3-2024-1@e042b5a7-d5af-4a47-98cc-20cb46d269fc")

Please enter your UNCode username: dmejiac
Please enter your password: ··········


Ejecute la siguiente celda para importar las librerías que permiten la ejecucuión de los queries de *Cassandra*.

In [4]:
!pip install cassandra-driver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 27.1 MB/s eta 0:00:00


In [5]:
import pandas as pd
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from IPython.display import display
import json

## **Conexión con Astra**
---

Realizaremos la conexión a la base de datos de [Astra](https://astra.datastax.com/) tal y cómo se mostró en el notebook `1_cassandra_python.ipynb`:

> **Nota**: No olvides cargar los archivos `bundle.zip` y `token.json`.

In [11]:
cloud_config= {
        "secure_connect_bundle": "bundle.zip" # nombre de su archivo de bundle
}
with open('token.json') as user_file:
  token = json.loads(user_file.read())

auth_provider = PlainTextAuthProvider(
        token["clientId"],
        token["secret"]
        )
cluster = Cluster(
        cloud=cloud_config,
        auth_provider=auth_provider
        )
session = cluster.connect()

FileNotFoundError: [Errno 2] No such file or directory: 'token.json'

## **Creación de la Base de Datos**
---

Primero, vamos a cargar el conjunto de datos desde `pandas` para subirlo al servidor de _Cassandra_:

In [12]:
df = pd.read_parquet("https://drive.google.com/uc?export=view&id=1TSP7sZBUDDJs0UVrvDv8MinGquXe0aE3")

El siguiente código crea las tablas necesarias para el taller, no debe modificar esta celda. Puede revisar el código si desea ver el proceso de creación de la base de datos y carga de datos.

In [ ]:
#@title **Carga de datos. Ejecute esta celda para obtener todos los datos**
queries = []
table_name = [
        "query_by_year",
        "query_by_country",
        "query_by_year_country"
        ]
queries.extend(
        [
            "DROP TABLE IF EXISTS test.{};".format(table)
            for table in table_name
            ]
        )
queries.append("""
CREATE TABLE IF NOT EXISTS test.query_by_year (
        show_id TEXT,
        type TEXT,
        title TEXT,
        director TEXT,
        country TEXT,
        date_added TEXT,
        release_year INT,
        rating TEXT,
        duration TEXT,
        listed_in TEXT,
        PRIMARY KEY(release_year, type, rating, show_id)
        )
;
"""
)

queries.append("""
CREATE TABLE IF NOT EXISTS test.query_by_country (
        show_id TEXT,
        type TEXT,
        title TEXT,
        director TEXT,
        country TEXT,
        date_added TEXT,
        release_year INT,
        rating TEXT,
        duration TEXT,
        listed_in TEXT,
        PRIMARY KEY(country, type, rating, show_id)
        )
;
""")

queries.append("""
CREATE TABLE IF NOT EXISTS test.query_by_year_country (
        show_id TEXT,
        type TEXT,
        title TEXT,
        director TEXT,
        country TEXT,
        date_added TEXT,
        release_year INT,
        rating TEXT,
        duration TEXT,
        listed_in TEXT,
        PRIMARY KEY((release_year, country), type, rating, show_id)
        )
;
""")

for query in queries:
    session.execute(query, timeout=None)

query_format = """
INSERT INTO
    test.{} (
            show_id, type, title, director, country, date_added,
            release_year, rating, duration, listed_in
            )
VALUES
    ('{}', '{}', '{}', '{}', '{}', '{}', {}, '{}', '{}', '{}')
;
"""

batch_size = 256
for table in table_name:
    data = [
            query_format.format(
                table, row.show_id, row.type, row.title,
                row.director, row.country, row.date_added,
                row.release_year, row.rating, row.duration,
                row.listed_in
                )
            for row in df.itertuples()
            ]
    for i in range(len(data) // batch_size + 1):
        batch_data = "\n".join(
                data[i * batch_size: (i + 1) * batch_size]
                )
        query = """
        BEGIN BATCH
        {}
        APPLY BATCH;
        """.format(batch_data)
        session.execute(query)

## **Recomendaciones**
---

* Para todos los ejercicios se consideran los datos de la tabla [Netflix Data: Cleaning, Analysis and Visualization](https://www.kaggle.com/datasets/ariyoomotade/netflix-data-cleaning-analysis-and-visualization), señalada al inicio del notebook. Además, las tablas se diseñaron siguiendo el enfoque **Query - First Design**.
* La base de datos creada tiene el esquema mostrado en la siguiente imagen (llaves de partición en rojo y llaves de agrupamiento en azul). En los ejercicios propuestos en este notebook deberá seleccionar la tabla más apropiada para una consulta en específico.

<img src="https://drive.google.com/uc?export=view&id=1AAtVRxVjKh_45Lb4ML4PDJRaqZDp4hox" width="40%">

## **1. Shows de un Año Específico**
---

En este primer ejercicio usted deberá completar la función `shows_year`, esta recibe como entrada un año y debe retornar una cadena de texto con una consulta **CQL** válida que permita obtener los campos `title` y `director` de **TODOS** los shows para un año de lanzamiento específico.

Debe modificar el string `query` para cumplir con la consulta. Se recomienda usar [f-strings](https://realpython.com/python-f-strings/) para poder cumplir con este requisito.

**Parámetros**

* `year`: entero que representa el año de lanzamiento de un show.

**Retorna**

* `query`: cadena de texto con una consulta válida de **CQL**.

In [13]:
def shows_year(year):
    query = """
    SELECT
        title, director
    FROM
        netflix.shows_by_year
    WHERE
        release_year = {year}
    ;
    """
    return query.format(year)

Use la siguiente celda para probar su código para filtrar los registros en el año 2018.

In [14]:
try:
    df = pd.DataFrame(session.execute(shows_year(2018)).all())
    display(df.head(10))
except Exception as e:
    print(e)

module '' has no attribute 'execute'


**Salida esperada:**

|    | title                        | director          |
|---:|:-----------------------------|:------------------|
|  0 | a champion heart             | david de vos      |
|  1 | pup star world tour          | robert vince      |
|  2 | ladylike                     | not given         |
|  3 | adventures in public school  | kyle rideout      |
|  4 | at eternitys gate            | julian schnabel   |
|  5 | operation finale             | chris weitz       |
|  6 | elliot the littlest reindeer | jennifer westcott |
|  7 | adrift                       | baltasar kormkur  |
|  8 | okkos inn                    | kitaro kosaka     |
|  9 | mirai                        | mamoru hosoda     |

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 1</b></font>
</summary>

Verifique que los campos seleccionados en su consulta están escritos correspondientemente a los nombres de los campos en las tablas y en minúscula:

  - título: `title`
  - director: `director`
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 2</b></font>
</summary>

Verifique que al momento de seleccionar la tabla la está escribiendo correctamente en su query, el nombre de la tabla debe cumplir la siguiente estructura:

```sql
<nombre_keyspace>.<nombre_tabla>
```
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 3</b></font>
</summary>

* Verifique que al momento de filtrar (momento de usar la sentencia `WHERE`) está usando una llave de partición o una llave de agrupamiento.
* Si la tabla sobre la que está haciendo la consulta, no tiene como llave de partición o agrupamiento el campo con el que está filtrando, probablemente está seleccionando mal la tabla.
</details>

### **Evaluar código**

In [15]:
grader.run_test("Test 1_1", globals())

Test 1_1


In [16]:
grader.run_test("Test 1_2", globals())

Test 1_2


In [17]:
grader.run_test("Test 1_3", globals())

Test 1_3


## **2. Shows de un Tipo y Año Específico**
---

En este punto debe seleccionar todos los shows de un tipo y año en específico. Debe completar la función `shows_year_type`, la cual deberá retornar una cadena de texto con una consulta **CQL** válida que contenga los campos `title` y `director` de cada una de las películas o series que cumplan con los argumentos dados.

**Parámetros**

* `year`: número entero que sirve para señalar el año de publicación original de la película o serie.
* `show_type`: string que indica el tipo del show que se desea obtener.

**Retorna**

* `query`: cadena de texto con una consulta válida para _Cassandra_.

In [ ]:
def shows_year_type(year, show_type):
    query = """
    SELECT
        title, director
    FROM
        netflix.shows_by_year_type
    WHERE
        release_year = {year} AND type = '{show_type}'
    ;
    """
    return query.format(year, show_type)

Use la siguiente celda para probar su código:

In [ ]:
try:
    df = pd.DataFrame(session.execute(shows_year_type(2017, "tv show")).all())
    display(df.head(10))
except Exception as e:
    print(e)

**Salida esperada:**

Para el el caso `shows_year_type(2017, "tv show")` se obtiene la siguiente tabla de ejemplo:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>title</th>
      <th>director</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>my secret romance</td>
      <td>not given</td>
    </tr>
    <tr>
      <th>1</th>
      <td>sons of the caliphate</td>
      <td>not given</td>
    </tr>
    <tr>
      <th>2</th>
      <td>because this is my first life</td>
      <td>not given</td>
    </tr>
    <tr>
      <th>3</th>
      <td>the bride of habaek</td>
      <td>not given</td>
    </tr>
    <tr>
      <th>4</th>
      <td>my sassy girl</td>
      <td>not given</td>
    </tr>
    <tr>
      <th>5</th>
      <td>tomorrow with you</td>
      <td>not given</td>
    </tr>
    <tr>
      <th>6</th>
      <td>chicago typewriter</td>
      <td>not given</td>
    </tr>
    <tr>
      <th>7</th>
      <td>march comes in like a lion</td>
      <td>not given</td>
    </tr>
    <tr>
      <th>8</th>
      <td>this evening</td>
      <td>not given</td>
    </tr>
    <tr>
      <th>9</th>
      <td>the kirlian frequency</td>
      <td>not given</td>
    </tr>
  </tbody>
</table>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 1</b></font>
</summary>

Puede usar el operador `AND` para unir dos condiciones lógicas. Por ejemplo, la comparación del año con la comparación del tipo.
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 2</b></font>
</summary>

Para realizar correctamente la consulta, recuerde los tipos de datos de los argumentos (`year` es de tipo `INT` y `show_type` de tipo `TEXT`). Recuerde que en **CQL** los datos de tipo texto deben ser acotados por comillas simples `''`.
</details>

### **Evaluar código**

In [ ]:
grader.run_test("Test 2_1", globals())

In [ ]:
grader.run_test("Test 2_2", globals())

## **3. Número de Shows por País y Tipo**
---

En este punto usted deberá completar la función `number_shows_country_type` la cual debe retornar una cadena de texto con una consulta **CQL** válida con el recuento de elementos por cada tipo de show en cada uno de los países. La tabla resultante debe obtener los campos `country` (país), `type` (tipo) y un campo adicional que debe nombrar como `counts` con el recuento de shows por tipo y país. Adicionalmente, debe retornar solo los valores del país indicado.

**Parámetros**

* `country`: string que indica el país de consulta.

**Retorna**

* `query`: cadena de texto con una consulta válida de **CQL**.

In [ ]:
def number_shows_country_type(country):
    query = """
    SELECT
        country, type, COUNT(*) AS counts
    FROM
        netflix.shows_by_country_type
    WHERE
        country = '{country}'
    GROUP BY
        country, type
    ;
    """
    return query.format(country)

Use la siguiente celda para probar su código:

In [ ]:
try:
    df = pd.DataFrame(session.execute(number_shows_country_type('germany')).all())
    display(df.head(10))
except Exception as e:
    print(e)

**Salida esperada:**

|    | country              | type    |   counts |
|---:|:---------------------|:--------|---------:|
|  0 | germany              | movie   |       79 |
|  1 | germany              | tv show |       25 |


In [ ]:
try:
    df = pd.DataFrame(session.execute(number_shows_country_type('poland')).all())
    display(df.head(10))
except Exception as e:
    print(e)

**Salida esperada:**

|    | country              | type    |   counts |
|---:|:---------------------|:--------|---------:|
|  0 | poland              | movie   |       22 |
|  1 | poland              | tv show |       8 |

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 1</b></font>
</summary>

Se recomienda revisar la función `COUNT()` tal y como se mostró en el taller guiado de _Cassandra_, recuerde que la sintaxis en el lenguaje *CQL* es muy similar al de `SQL`.
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 2</b></font>
</summary>
    
Verifique que los campos seleccionados en su consulta corresponden a los nombres de los campos en las tablas y se encuentran en minúscula:

  * país: `country`
  * tipo: `type`

Igualmente recuerde renombrar los conteos obtenidos de la función `COUNT()`, algo de la forma:

```sql
COUNT(columna) AS counts
```
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 3</b></font>
</summary>

Verifique que esté usando un esquema válido, es decir:

```sql
SELECT
    ...
FROM
    test.<nombre_tabla>
...
```
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 4</b></font>
</summary>

Al igual que en el punto anterior, debe realizar una consulta correcta, teniendo en cuenta el tipo de dato de `country` es un string
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 5</b></font>
</summary>

No olvide que para realizar un `GROUP BY` sobre una llave de agrupamiento, debe agrupar sobre todas las llaves de agrupamiento anteriores a esta, es decir, de acuerdo a las posiciones en las que fue creada la tabla.
</details>

### **Evaluar código**

In [ ]:
grader.run_test("Test 3_1", globals())

In [ ]:
grader.run_test("Test 3_2", globals())

## **4. Shows en un Año y País Específico**
---

En este ejercicio usted debe completar la función `shows_year_country`, la cual deberá retornar una cadena de texto con una **CQL** válida para obetener los campos `title` y `director` de todas las películas en un año y país específico.

**Parámetros**

* `year`: número entero que indica el año sobre el que se debe filtrar.
* `country`: string que representa el país de producción de la serie o película que se desea filtrar.

**Retorna**

* `query`: cadena de texto con una consulta válida para _Cassandra_.

In [ ]:
def shows_year_country(year, country):
    query = """
    SELECT
        title, director
    FROM
        netflix.shows_by_year_country
    WHERE
        release_year = {year} AND country = '{country}'
    ;
    """
    return query.format(year, country)

Use la siguiente celda para probar su código:

In [ ]:
try:
    df = pd.DataFrame(session.execute(shows_year_country(2015, 'germany')).all())
    display(df.head(10))
except Exception as e:
    print(e)

**Salida esperada:**

Para el caso `shows_year_country(2015, 'germany')` debe obtener:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>title</th>
      <th>director</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>look whos back</td>
      <td>david wnendt</td>
    </tr>
    <tr>
      <th>1</th>
      <td>colonia</td>
      <td>florian gallenberger</td>
    </tr>
    <tr>
      <th>2</th>
      <td>pedal the world</td>
      <td>felix starck</td>
    </tr>
    <tr>
      <th>3</th>
      <td>a heavy heart</td>
      <td>thomas stuber</td>
    </tr>
    <tr>
      <th>4</th>
      <td>trken  ein baby</td>
      <td>sinan akku</td>
    </tr>
    <tr>
      <th>5</th>
      <td>ho mermaid adventures</td>
      <td>not given</td>
    </tr>
  </tbody>
</table>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 1</b></font>
</summary>

Para realizar correctamente la consulta, recuerde los tipos de datos de los argumentos (`year` es de tipo `INT` y `country` de tipo `TEXT`). Recuerde que en **CQL** los datos de tipo texto deben ser acotados por comillas simples `''`.
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 2</b></font>
</summary>

Puede usar el método `.format` para sustituir variables dentro del string de una consulta `query.format(arg1, arg2, ...)` siempre que tenga un par de llaves  `{}` dentro del string.
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 3</b></font>
</summary>

* Verifique que al momento de filtrar (momento de usar la sentencia `WHERE`) está usando una llave de partición o una llave de agrupamiento.
* Si la tabla sobre la que está haciendo la consulta, no tiene como llave de partición o agrupamiento el campo con el que está filtrando, probablemente está seleccionando mal la tabla.
</details>

### **Evaluar código**

In [ ]:
grader.run_test("Test 4_1", globals())

In [ ]:
grader.run_test("Test 4_2", globals())

In [ ]:
grader.run_test("Test 4_3", globals())

## **5. Conteo de Rating de Shows por País**
---

En este punto deberá implementar la función `number_shows_rating_country`, la cual debe retornar un string con una consulta **CQL** válida que retorne la cantidad de shows que hay por tipo rating en cada país. La consulta debe extraer los campos `country` (país), `rating` (puntuación) y un campo adicional `counts` que debe contener el recuento de shows. Adicionalmente, debe retornar solo los valores del país indicado.

**Parámetros**

* `country`: string que indica el país de consulta.

**Retorna**

* `query`: cadena de texto con una consulta válida de **CQL**.

In [ ]:
def number_shows_rating_country(country):
    query = """
    SELECT
        country, rating, COUNT(*) AS counts
    FROM
        netflix.shows_by_rating_country
    WHERE
        country = '{country}'
    GROUP BY
        country, rating
    ;
    """
    return query.format(country)

Use la siguiente celda para probar su código:

In [ ]:
try:
    df = pd.DataFrame(session.execute(number_shows_rating_country('germany')).all())
    display(df.head(10))
except Exception as e:
    print(e)

**Salida esperada:**

|    | country   | rating   |   counts |
|---:|:----------|:---------|---------:|
|  0 | germany   | g        |        1 |
|  1 | germany   | nr       |        1 |
|  2 | germany   | pg       |       11 |
|  3 | germany   | r        |       13 |
|  4 | germany   | tv       |       12 |
|  5 | germany   | tvg      |        4 |
|  6 | germany   | tvma     |       30 |
|  7 | germany   | tvpg     |        5 |
|  8 | germany   | tvy      |        2 |
|  9 | germany   | tv       |        3 |

In [ ]:
try:
    df = pd.DataFrame(session.execute(number_shows_rating_country('poland')).all())
    display(df.head(10))
except Exception as e:
    print(e)

**Salida esperada:**

|    | country   | rating   |   counts |
|---:|:----------|:---------|---------:|
|  0 | poland   | tv       |        2 |
|  1 | poland   | tvma       |       19 |
|  2 | poland   | tvpg       |        1 |
|  3 | poland   | tvma        |       8 |


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 1</b></font>
</summary>

Se recomienda revisar la función `COUNT()` tal y como se mostró en el taller guiado de _Cassandra_, recuerde que la sintaxis en el lenguaje *CQL* es muy similar al de `SQL`.
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 2</b></font>
</summary>

Verifique que los campos seleccionados en su consulta corresponden a los nombres de los campos en las tablas y se encuentran en minúscula:

  * país: `country`
  * rating: `rating`

Igualmente recuerde renombrar los conteos obtenidos de la función `COUNT()`, algo de la forma:

```sql
COUNT(columna) AS counts
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 3</b></font>
</summary>

Verifique que esté usando un esquema válido, es decir:

```sql
SELECT
    ...
FROM
    test.<nombre_tabla>
...
```
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 4</b></font>
</summary>

No olvide que para hacer una agrupación en **`CQL`** debe usar todas las claves de partición y las claves de agrupamiento posicionalmente anteriores (de acuerdo al orden en el que se creó la tabla).
</details>

### **Evaluar código**

In [ ]:
grader.run_test("Test 5_1", globals())

In [ ]:
grader.run_test("Test 5_2", globals())

## **6. Conteo de Rating de Shows por Año**
---

En este ejercicio usted debe completar la función `number_shows_rating_year` la cual deberá retornar una cadena de texto con una consulta **CQL** válida que contenga los campos `release_year`, `type`, `rating` y `counts` (la cantidad total de películas o series por año que tienen una misma clasificación). Adicionalmente, debe retornar solo los valores con el año igual al indicado.

**Parámetros**

 * `year`: número entero que sirve para señalar el año de publicación original de la película o serie.

**Retorna**

* `query`: cadena de texto con una consulta válida para _Cassandra_.

In [ ]:
def number_shows_rating_year(year):
    query = """
    SELECT
        release_year, type, rating, COUNT(*) AS counts
    FROM
        netflix.shows_by_rating_year
    WHERE
        release_year = {year}
    GROUP BY
        release_year, type, rating
    ;
    """
    return query.format(year)

Use la siguiente celda para probar su código:

In [ ]:
try:
    df = pd.DataFrame(session.execute(number_shows_rating_year(2000)).all())
    display(df.head(10))
except Exception as e:
    print(e)

**Salida esperada:**

|    | release_year   | type   | rating   |   counts |
|---:|:---------------|:-------|---------:|---------:|
|  0 | 2000           |movie   | g        |        2 |
|  1 | 2000           |movie   | pg       |        14 |
|  2 | 2000           |movie   | r       |       8 |
|  3 | 2000           |movie   | tv        |       5 |
|  4 | 2000           |movie   | tvma       |       1 |
|  5 | 2000           |movie   | tvpg      |        3 |
|  6 | 2000           |tv show   | tvpg     |       2 |
|  7 | 2000           |tv show   | tvy     |        2 |

In [ ]:
try:
    df = pd.DataFrame(session.execute(number_shows_rating_year(1990)).all())
    display(df.head(10))
except Exception as e:
    print(e)

**Salida esperada:**

|    | release_year   | type   | rating   |   counts |
|---:|:---------------|:-------|---------:|---------:|
|  0 | 1990           |movie   | pg        |        5 |
|  1 | 1990           |movie   | r       |        4 |
|  2 | 1990           |movie   | tv       |       7 |
|  3 | 1990           |movie   | tvma        |       3 |
|  4 | 1990           |tv show   | tv       |       2 |
|  5 | 1990           |tv show   | tvpg      |        1 |


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 1</b></font>
</summary>

Los campos que se consideren en la función `GROUP BY` deben estar contenidos en la llave primaria y deben enunciarse en el mismo orden en el que se declararon al momento de crear la tabla, de lo contrario el resultado será una consulta inválida.
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 2</b></font>
</summary>

Al igual que en las bases de datos relacionales, existe la palabra reservada `AS` para crear un alias y así renombrar una columna o tabla al realizar una consulta.
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 3</b></font>
</summary>

La función `COUNT` añade una nueva columna al resultado del query. La misma debe ser renombrada a `counts` para que coincida con la salida esperada de ejemplo.
</details>

### **Evaluar código**

In [ ]:
grader.run_test("Test 6_1", globals())

In [ ]:
grader.run_test("Test 6_2", globals())

## **7. Conteo de Ratings de Shows por Año y País**
---

En este punto usted deberá completar la función `number_shows_rating_year_country`, la cual debe retornar una cadena de texto con una consulta **CQL** válida que permita obtener la cantidad de shows que hay por rating, en cada país y por año. La consulta debe extraer los campos: `release_year` (año de lanzamiento), `country` (país), `rating` (puntuación) y `counts` (recuento de valores). Adicionalmente, debe retornar solo los valores con el año y país igual al indicado.

**Parámetros**

* `year`: número entero que sirve para señalar el año de publicación original de la película o serie.

* `country`: string que indica el país de consulta.

**Retorna**

* `query`: cadena de texto con una consulta válida de **CQL**.

In [ ]:
def number_shows_rating_year_country(year, country):
    query = """
    SELECT
        release_year, country, rating, COUNT(*) AS counts
    FROM
        netflix.shows_by_rating_year_country
    WHERE
        release_year = {year} AND country = '{country}'
    GROUP BY
        release_year, country, rating
    ;
    """
    return query.format(year, country)

Use la siguiente celda para probar su código:

In [ ]:
try:
    df = pd.DataFrame(session.execute((number_shows_rating_year_country(2018, 'russia'))).all())
    display(df.head(10))
except Exception as e:
    print(e)

**Salida esperada:**

|    |   release_year | country        | rating   |   counts |
|---:|---------------:|:---------------|:---------|---------:|
|  0 |           2018 | russia         | pg       |        1 |
|  1 |           2018 | russia         | tvma     |        1 |
|  2 |           2018 | russia         | tvy      |        1 |


In [ ]:
try:
    df = pd.DataFrame(session.execute((number_shows_rating_year_country(2020, 'germany'))).all())
    display(df.head(10))
except Exception as e:
    print(e)

**Salida esperada:**

|    |   release_year | country        | rating   |   counts |
|---:|---------------:|:---------------|:---------|---------:|
|  0 |           2020 | germany        | tv       |        1 |
|  1 |           2020 | germany        | tvg      |        1 |
|  2 |           2020 | germany        | tvma     |        9 |
|  3 |           2020 | germany        | tvy      |        2 |
|  4 |           2020 | germany        | tv       |        2 |
|  5 |           2020 | germany        | tvma     |        8 |
|  6 |           2020 | germany        | tvpg     |        1 |


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 1</b></font>
</summary>

Se recomienda revisar la función `COUNT()` tal y como se mostró en el taller guiado de _Cassandra_, recuerde que la sintaxis en el lenguaje *CQL* es muy similar al de `SQL`.
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 2</b></font>
</summary>

Verifique que los campos seleccionados en su consulta están escritos correspondientemente a los nombres de los campos en las tablas y en minúscula:

  - año de lanzamiento: `release_year`
  - país: `country`
  - rating: `rating`
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 3</b></font>
</summary>

Verifique que al momento de seleccionar la tabla la está escribiendo correctamente en su query, el nombre de la tabla debe cumplir la siguiente estructura:

```sql
<nombre_keyspace>.<nombre_tabla>
```
</details>

### **Evaluar código**

In [ ]:
grader.run_test("Test 7_1", globals())

In [ ]:
grader.run_test("Test 7_2", globals())

# Evaluación
---

In [ ]:
grader.submit_task(globals())

# Créditos
---

**Profesor**

- [Jorge E. Camargo, PhD](https://dis.unal.edu.co/~jecamargom/)

**Diseño, desarrollo del notebook y material audiovisual**

- [Juan S. Lara MSc](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/)

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*